<a href="https://colab.research.google.com/github/trungduyen0220/text-classification-assignment/blob/master/Assignment2_NguyenThiMyDuyen_MSE13028.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programming Assignment 2: Vietnamese Word Segmentation

Nguyễn Thị Mỹ Duyên - 19MSE13028

## Getting the data

In [1]:
!rm -f train.txt
!rm -f test.txt
!wget https://www.dl.dropboxusercontent.com/s/reor8jnqedk7svt/train.txt
!wget https://www.dl.dropboxusercontent.com/s/zp635cd1zhofm62/test.txt

--2020-09-18 22:09:41--  https://www.dl.dropboxusercontent.com/s/reor8jnqedk7svt/train.txt
Resolving www.dl.dropboxusercontent.com (www.dl.dropboxusercontent.com)... 162.125.1.15, 2620:100:6016:15::a27d:10f
Connecting to www.dl.dropboxusercontent.com (www.dl.dropboxusercontent.com)|162.125.1.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5485066 (5.2M) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>]   5.23M  34.4MB/s    in 0.2s    

2020-09-18 22:09:42 (34.4 MB/s) - ‘train.txt’ saved [5485066/5485066]

--2020-09-18 22:09:42--  https://www.dl.dropboxusercontent.com/s/zp635cd1zhofm62/test.txt
Resolving www.dl.dropboxusercontent.com (www.dl.dropboxusercontent.com)... 162.125.1.15, 2620:100:6016:15::a27d:10f
Connecting to www.dl.dropboxusercontent.com (www.dl.dropboxusercontent.com)|162.125.1.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 871524 (851K) [text/plain]
Saving to: ‘test.txt’

test.tx

Firstly, import necessary packages for the work.

In [2]:
!pip install sklearn-crfsuite==0.3.6
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

## Loading data

In [3]:
def load_data(input_file):    
    sentence = []
    with open(input_file, "r") as f:
        for line in f:
            line = line.strip()
            if line == "":
                continue
            wordtags = line.split()
            word = wordtags[0], wordtags[1]
            sentence.append(word)
    return sentence

train_data = load_data('./train.txt')
test_data = load_data('./test.txt')
train_data[0:10]

[('Nam', 'B-W'),
 ('hồn', 'B-W'),
 ('nhiên', 'I-W'),
 (':', 'B-W'),
 ('"', 'B-W'),
 ('Tụi', 'B-W'),
 ('tôi', 'B-W'),
 ('xài', 'B-W'),
 ('tiền', 'B-W'),
 ('ngân', 'B-W')]

## Features

In [4]:
def word2features(sentence, i):
    """
    Arguments:
        sentence (list): list of words [w1, w2,...,w_n]
        i (int): index of the word
    Return:
        features (dict): dictionary of features
    """
    word = sentence[i]
    features = {
        'is_first': i == 0,
        'is_last': i == len(sentence) - 1,
        'is_first_capital': word[0].isupper(),
        'is_all_caps': int(word.upper() == word),
        'is_all_lower': word.lower() == word,
        'word': word,
        'word.lower()': word.lower(),
        'prefix_1': word[0],
        'prefix_2': word[:2],
        'prefix_3': word[:3],
        'prefix_4': word[:4],
        'suffix_1': word[-1],
        'suffix_2': word[-2:],
        'suffix_3': word[-3:],
        'suffix_4': word[-4:],
        'prev_word': '' if i==0 else sentence[i-1].lower(),
        'next_word': '' if i==len(sentence)-1 else sentence[i+1].lower(),
        'has_hyphen': '-' in word,
        'is_numeric': word.isdigit(),
        'capitals_inside': word[1:].lower() != word[1:]
    }
    return features
                                             
def sent2features(sentence):
    return [word2features(sentence, i) for i in range(len(sentence))]

def sent2labels(sentence):
    return [postag for token, postag in sentence]

def untag(sentence):
    return [token for token, _ in sentence]

Check the first word to see it's features.

In [5]:
sent2features(untag(train_data))[0]

{'capitals_inside': False,
 'has_hyphen': False,
 'is_all_caps': 0,
 'is_all_lower': False,
 'is_first': True,
 'is_first_capital': True,
 'is_last': False,
 'is_numeric': False,
 'next_word': 'hồn',
 'prefix_1': 'N',
 'prefix_2': 'Na',
 'prefix_3': 'Nam',
 'prefix_4': 'Nam',
 'prev_word': '',
 'suffix_1': 'm',
 'suffix_2': 'am',
 'suffix_3': 'Nam',
 'suffix_4': 'Nam',
 'word': 'Nam',
 'word.lower()': 'nam'}

Extract features from the data.

In [6]:
X_train = [sent2features(untag(train_data))]
y_train = [sent2labels(train_data)]

X_test = [sent2features(untag(test_data))]
y_test = [sent2labels(test_data)]

## Training

Here, I use the model which is optimised by Gradient Descent using the LBGS method with L1 and L2 regularisation. 

In [7]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.01,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    all_possible_states=True,
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=True, all_possible_transitions=True,
    averaging=None, c=None, c1=0.01, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

## Evaluation Measures

Now I will evaluate the trained CRF model on the test data.

- P(recision): (number of words correctly segmented)/(number of words in the system output)
- R(ecall): (number of words correctly segmented)/(number of words in the reference corpus)
- F1 measure = 2*P*R/(P+R)

In [8]:
y_pred = crf.predict(X_test)
print(metrics.flat_accuracy_score(y_test, y_pred))

0.9524130538088345


In [9]:
print(metrics.flat_classification_report(
    y_test, y_pred, labels=crf.classes_, digits=3
))

              precision    recall  f1-score   support

         B-W      0.954     0.978     0.966     62131
         I-W      0.949     0.896     0.922     28356

    accuracy                          0.952     90487
   macro avg      0.952     0.937     0.944     90487
weighted avg      0.952     0.952     0.952     90487



## Evaluation code

Use the package [seqeval](https://github.com/chakki-works/seqeval) for evaluation.

In [10]:
!pip install seqeval[cpu]

In [11]:
from seqeval.metrics import precision_score, recall_score, f1_score

def get_tags(filepath):
    res = []
    with open(filepath, 'r') as f:
        cur_sen = []
        for line in f:
            line = line.strip()
            if line == '':
                if len(cur_sen) != 0:
                    res.append(cur_sen)
                    cur_sen = []
            else:
                word, tag = line.split()
                cur_sen.append(tag)
    if len(cur_sen) != 0:
        res.append(cur_sen)
    return res

def evaluate(test_file, output_file):
    y_true = get_tags(test_file)
    y_pred = get_tags(output_file)

    p = precision_score(y_true, y_pred)
    r = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return p, r, f1

Create two file output.txt and answer.txt and evaluate them with p, r and f1.

In [12]:
def extract_file(filename, data):
  with open(filename, 'w+') as filehandle:
    for s in data:
      word_output, tag_output = s[0], s[1]
      filehandle.writelines(word_output + "\t" + tag_output + "\n")

In [13]:
# Sentence with predict tag
sentence = []
for i, index in enumerate(test_data):
  word_output, tag_output = index[0], y_pred[0][i]
  word = word_output, tag_output
  sentence.append(word)

In [14]:
extract_file('output_file.txt', sentence)
extract_file('test_file.txt', test_data)
evaluate('output_file.txt', 'test_file.txt')

(0.9276367674751734, 0.904433111023931, 0.9158879989829647)